In [ ]:
!wget https://motchallenge.net/data/MOT17.zip

--2025-01-03 19:31:26--  https://motchallenge.net/data/MOT17.zip
Resolving motchallenge.net (motchallenge.net)... 131.159.19.34, 2a09:80c0:18::1034
Connecting to motchallenge.net (motchallenge.net)|131.159.19.34|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 5860214001 (5.5G) [application/zip]
Saving to: ‘MOT17.zip’

MOT17.zip           100%[===================>]   5.46G  16.3MB/s    in 5m 44s  

2025-01-03 19:37:12 (16.3 MB/s) - ‘MOT17.zip’ saved [5860214001/5860214001]



In [ ]:
!unzip -qq MOT17.zip

In [ ]:
!pip install ultralytics -q

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/905.3 kB 51.2 MB/s eta 0:00:00


In [ ]:
import pandas as pd
import os
import yaml
import shutil
import configparser
import ultralytics
ultralytics.checks()

from tqdm import tqdm
from ultralytics import YOLO

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Setup complete ✅ (2 CPUs, 12.7 GB RAM, 43.8/112.6 GB disk)


In [ ]:
def convert_to_yolo_format (bb, img_width, img_height):
    x_center = bb['bb_left'] + (bb['bb_width'] / 2)
    y_center = bb['bb_top'] + (bb['bb_height'] / 2)

    # Normalize the coordinates by the dimensions of the image
    x_center /= img_width
    y_center /= img_height
    bb_width_normalized = bb['bb_width'] / img_width
    bb_height_normalized = bb['bb_height'] / img_height

    # Clip the values to make sure they are between 0 and 1
    x_center = max(min(x_center, 1), 0)
    y_center = max(min(y_center, 1), 0)
    bb_width_normalized = max(min(bb_width_normalized, 1), 0)
    bb_height_normalized = max(min(bb_height_normalized, 1), 0)

    return (x_center, y_center, bb_width_normalized, bb_height_normalized)

In [ ]:
def process_folder (folder_path):
  # Read image dimensions from seqinfo.ini
   config = configparser.ConfigParser()
   config.read(os.path.join(folder_path, 'seqinfo.ini'))
   img_width = int(config['Sequence']['imWidth'])
   img_height = int(config['Sequence']['imHeight'])

   # Load ground truth data
   gt_path = os.path.join(folder_path, 'det/det.txt')
   gt_data = pd.read_csv(
        gt_path,
        header=None,
        names=['frame', 'id', 'bb_left', 'bb_top', 'bb_width', 'bb_height', 'conf', 'class', 'visibility']
    )

   labels_folder = os.path.join(folder_path, 'labels')
   os.makedirs(labels_folder, exist_ok=True)

   for frame_number in gt_data['frame'].unique():
      # data in frame = frame_number
      frame_data = gt_data[gt_data['frame'] == frame_number]
      label_file = os.path.join(labels_folder, f'{frame_number:06d}.txt')

      with open(label_file, 'w') as file:
            for _, row in frame_data.iterrows():
                yolo_bb = convert_to_yolo_format(row, img_width, img_height)
                file.write(f'0 {yolo_bb[0]} {yolo_bb[1]} {yolo_bb[2]} {yolo_bb[3]}\n')

In [ ]:
def process_all_folders(base_directory):
    # List all subdirectories in the base directory
    for folder_name in tqdm(os.listdir(base_directory)):
        folder_path = os.path.join(base_directory, folder_name)

        # Delete folder not contain 'FRCNN' in name
        if 'FRCNN' not in folder_name:
          if 'DPM' not in folder_name:
            os.system(f'rm -rf {folder_path}')
            continue

        if os.path.isdir(folder_path):
            process_folder(folder_path)

In [ ]:
process_all_folders('/content/MOT17/train')
process_all_folders('/content/MOT17/test')

100%|██████████| 21/21 [00:17<00:00,  1.21it/s]


In [ ]:
def rename_and_move_files(src_folder, dst_folder, folder_name, file_extension):
    for filename in os.listdir(src_folder):
        if filename.endswith(file_extension):
            # Include folder name in the new filename
            new_filename = f'{folder_name}_{filename}'
            shutil.move(os.path.join(src_folder, filename), os.path.join(dst_folder, new_filename))

In [ ]:
def move_files_all_folders(base_directory):
    images_dir = os.path.join(base_directory, 'images')
    labels_dir = os.path.join(base_directory, 'labels')
    os.makedirs(images_dir, exist_ok=True)
    os.makedirs(labels_dir, exist_ok=True)

    for folder_name in tqdm(os.listdir(base_directory)):
        if folder_name in ['images', 'labels']:  # Skip these folders
            continue

        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path):
            rename_and_move_files(os.path.join(folder_path, 'img1'), images_dir, folder_name, '.jpg')
            rename_and_move_files(os.path.join(folder_path, 'labels'), labels_dir, folder_name, '.txt')

In [ ]:
move_files_all_folders('/content/MOT17/train')
move_files_all_folders('/content/MOT17/test')

100%|██████████| 16/16 [00:00<00:00, 29.87it/s]


In [ ]:
move_files_all_folders('MOT17/train')
move_files_all_folders('MOT17/test')

100%|██████████| 16/16 [00:00<00:00, 16602.89it/s]


In [ ]:
def delete_subfolders(base_directory):
    for folder_name in os.listdir(base_directory):
        folder_path = os.path.join(base_directory, folder_name)
        if os.path.isdir(folder_path) and folder_name not in ['images', 'labels']:
            shutil.rmtree(folder_path)
            print(f"Deleted folder: {folder_name}")

In [ ]:
# Delete all subfolders except 'images' and 'labels'
delete_subfolders('MOT17/train')
delete_subfolders('MOT17/test')

Deleted folder: MOT17-10-DPM
Deleted folder: MOT17-09-DPM
Deleted folder: MOT17-11-FRCNN
Deleted folder: MOT17-05-DPM
Deleted folder: MOT17-13-FRCNN
Deleted folder: MOT17-04-FRCNN
Deleted folder: MOT17-11-DPM
Deleted folder: MOT17-09-FRCNN
Deleted folder: MOT17-02-FRCNN
Deleted folder: MOT17-10-FRCNN
Deleted folder: MOT17-02-DPM
Deleted folder: MOT17-04-DPM
Deleted folder: MOT17-13-DPM
Deleted folder: MOT17-05-FRCNN
Deleted folder: MOT17-06-FRCNN
Deleted folder: MOT17-03-FRCNN
Deleted folder: MOT17-08-FRCNN
Deleted folder: MOT17-07-DPM
Deleted folder: MOT17-07-FRCNN
Deleted folder: MOT17-12-DPM
Deleted folder: MOT17-14-FRCNN
Deleted folder: MOT17-01-FRCNN
Deleted folder: MOT17-03-DPM
Deleted folder: MOT17-06-DPM
Deleted folder: MOT17-08-DPM
Deleted folder: MOT17-14-DPM
Deleted folder: MOT17-01-DPM
Deleted folder: MOT17-12-FRCNN


In [ ]:
os.listdir('/content/MOT17/test/images')

['MOT17-14-FRCNN_000670.jpg',
 'MOT17-03-DPM_001493.jpg',
 'MOT17-01-DPM_000074.jpg',
 'MOT17-03-FRCNN_000188.jpg',
 'MOT17-14-FRCNN_000341.jpg',
 'MOT17-12-FRCNN_000696.jpg',
 'MOT17-06-FRCNN_000242.jpg',
 'MOT17-03-DPM_000591.jpg',
 'MOT17-12-DPM_000765.jpg',
 'MOT17-08-FRCNN_000032.jpg',
 'MOT17-06-DPM_000901.jpg',
 'MOT17-03-FRCNN_000353.jpg',
 'MOT17-03-FRCNN_000802.jpg',
 'MOT17-08-FRCNN_000560.jpg',
 'MOT17-12-DPM_000582.jpg',
 'MOT17-03-DPM_000133.jpg',
 'MOT17-14-DPM_000295.jpg',
 'MOT17-03-DPM_000274.jpg',
 'MOT17-03-DPM_000592.jpg',
 'MOT17-06-FRCNN_000181.jpg',
 'MOT17-01-DPM_000398.jpg',
 'MOT17-06-DPM_001003.jpg',
 'MOT17-03-DPM_000687.jpg',
 'MOT17-12-FRCNN_000807.jpg',
 'MOT17-03-FRCNN_000612.jpg',
 'MOT17-03-FRCNN_001222.jpg',
 'MOT17-08-FRCNN_000499.jpg',
 'MOT17-06-DPM_000196.jpg',
 'MOT17-12-FRCNN_000615.jpg',
 'MOT17-08-DPM_000184.jpg',
 'MOT17-14-DPM_000728.jpg',
 'MOT17-01-DPM_000133.jpg',
 'MOT17-06-FRCNN_000660.jpg',
 'MOT17-03-DPM_001418.jpg',
 'MOT17-14-DPM_0

In [ ]:
class_labels = [
    'objects'
]
dataset_root_dir = os.path.join(
    os.getcwd(),
    'MOT17'
)
yolo_yaml_path = os.path.join(
    dataset_root_dir,
    'mot17_data.yml'
)

data_yaml = {
    'path': dataset_root_dir,
    'train': 'train/images',
    'val': 'test/images',
    'nc': len(class_labels),
    'names': class_labels
}

with open(yolo_yaml_path, 'w') as f:
    yaml.dump(data_yaml, f, default_flow_style=False)

In [ ]:
from ultralytics import YOLO

# Load the YOLOv8 model
model = YOLO('yolov8s.pt')

# Config
epochs = 30
batch_size = -1 # Auto scale based on GPU memory
img_size = 640
project_name = 'models/yolo'
name = 'yolov8s'

# Train the model
results = model.train(
    data=yolo_yaml_path,
    epochs=epochs,
    batch=batch_size,
    imgsz=img_size,
    project=project_name,
    name=name
)

100%|██████████| 21.5M/21.5M [00:00<00:00, 362MB/s]


Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
engine/trainer: task=detect, mode=train, model=yolov8s.pt, data=/content/MOT17/mot17_data.yml, epochs=30, time=None, patience=100, batch=-1, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=models/yolo, name=yolov8s, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True

100%|██████████| 755k/755k [00:00<00:00, 91.5MB/s]


Overriding model.yaml nc=80 with nc=1

                   from  n    params  module                                       arguments                     
  0                  -1  1       928  ultralytics.nn.modules.conv.Conv             [3, 32, 3, 2]                 
  1                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  2                  -1  1     29056  ultralytics.nn.modules.block.C2f             [64, 64, 1, True]             
  3                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  4                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  5                  -1  1    295424  ultralytics.nn.modules.conv.Conv             [128, 256, 3, 2]              
  6                  -1  2    788480  ultralytics.nn.modules.block.C2f             [256, 256, 2, True]           
  7                  -1  1   1180672  ultralytics

100%|██████████| 5.35M/5.35M [00:00<00:00, 295MB/s]


...

                   all      11838     110141      0.465      0.773      0.455      0.326

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      30/30      11.8G     0.5179     0.8725      0.816        233        640: 100%|██████████| 201/201 [04:40<00:00,  1.40s/it]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 112/112 [02:40<00:00,  1.43s/it]


                   all      11838     110141      0.465      0.778      0.454      0.326

30 epochs completed in 3.762 hours.
Optimizer stripped from models/yolo/yolov8s/weights/last.pt, 22.5MB
Optimizer stripped from models/yolo/yolov8s/weights/best.pt, 22.5MB

Validating models/yolo/yolov8s/weights/best.pt...
Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 112/112 [02:58<00:00,  1.60s/it]


                   all      11838     110141      0.465      0.773      0.455      0.326
Speed: 0.1ms preprocess, 3.1ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to models/yolo/yolov8s


In [ ]:
from ultralytics import YOLO

# Load the trained model
model_path = '/content/models/yolo/yolov8s/weights/best.pt'
model = YOLO(model_path)

metrics = model.val(
    project=project_name,
    name='detect/val'
)

Ultralytics 8.3.57 🚀 Python-3.10.12 torch-2.5.1+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 11,125,971 parameters, 0 gradients, 28.4 GFLOPs


val: Scanning /content/MOT17/test/labels.cache... 5908 images, 5930 backgrounds, 0 corrupt: 100%|██████████| 11838/11838 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 740/740 [03:25<00:00,  3.60it/s]


                   all      11838     110141      0.465      0.774      0.454      0.326
Speed: 0.2ms preprocess, 5.6ms inference, 0.0ms loss, 2.0ms postprocess per image
Results saved to models/yolo/detect/val


In [ ]:
sample_path = '/content/MOT17/test/images/MOT17-01-FRCNN_000004.jpg'
results = model.predict(
    sample_path,
    project=project_name,
    name='detect/predict',
    save=True
)


image 1/1 /content/MOT17/test/images/MOT17-01-FRCNN_000004.jpg: 384x640 6 objectss, 46.3ms
Speed: 5.7ms preprocess, 46.3ms inference, 1.6ms postprocess per image at shape (1, 3, 384, 640)
Results saved to models/yolo/detect/predict


In [ ]:
from ultralytics import YOLO

# Load the trained model
model_path = '/kaggle/working/models/yolo/yolov8s_mot17_det2/weights/best.pt'
model = YOLO(model_path)

metrics = model.val(
    project=project_name,
    name='detect/val'
)